In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.getcwd()

'/Users/chuqiwang/Desktop/US_PFAS_Project/Illinois'

In [3]:
il_df1 = pd.read_excel('Export_Scott AFB.xlsx')

In [4]:
il_df1

,Component ID,Component,State,Installation Name,Installation Type,Lab Report Id,Laboratory Sample ID,Sample Date,Analysis Method,CAS Number,Analyte Abbreviation,Final Result,Limit of Detection,Final Units,Final Qualification,Treatment System,Pre Or Post Treatment
0,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,757124-72-4,4:2FTS,NaN,8.0,ng/L,U,No,NaN
1,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,27619-97-2,6:2FTS,NaN,8.0,ng/L,U,No,NaN
2,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,39108-34-4,8:2FTS,NaN,8.0,ng/L,U,No,NaN
3,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,2991-50-6,NEtFOSAA,NaN,8.0,ng/L,U,No,NaN
4,5700,Air Force,Illinois,Scott AFB,Active,FA91647,FA91647-1,12/12/2021,QSM_B15,2355-31-9,NMeFOSAA,NaN,8.0,ng/L,U,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,2706-90-3,PFPeA,NaN,4.0,ng/L,U,No,NaN
452,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,2706-91-4,PFPeS,NaN,4.0,ng/L,U,No,NaN
453,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,376-06-7,PFTeDA,NaN,4.0,ng/L,U,No,NaN
454,5700,Air Force,Illinois,Scott AFB,Active,FC14593,FC14593-2,4/3/2024,QSM_B15,72629-94-8,PFTrDA,NaN,4.0,ng/L,U,No,NaN


In [5]:
columns = {
    'sample_id': str,
    'sampling_location': str,
    'data_source': str,
    'latitude': float,
    'longitude': float,
    'sampling_date': 'datetime64[ns]',
    'pws_id': str,
    'water_system_name': str,
    'population': int,
    'well_type': str,
    'well_depth': float,
    'units': str,
    'source_type': str,
    'state': str,
    'city': str,
    'county': str,
    'MRL': float,
    'DL': float,
    'analytical_methods': str,
    'is_detected': bool,
    'analyte': str,
    'concentration': float,
    'pfas_total': float,
    'note': str   
}

In [6]:
df1 = pd.DataFrame({col: pd.Series(dtype=dt) for col, dt in columns.items()})
df1['analyte'] = il_df1['Analyte Abbreviation'].str.upper()
df1['concentration'] = il_df1['Final Result'].fillna(0)
df1['sampling_location'] = il_df1['Installation Name']
df1['DL'] = il_df1['Limit of Detection']
df1['sampling_date'] = pd.to_datetime(il_df1['Sample Date'])
df1['sample_id'] = il_df1['Laboratory Sample ID']
df1['units'] = il_df1['Final Units']
df1['state'] = il_df1['State']
df1['analytical_methods'] = il_df1['Analysis Method']
df1['data_source'] = 'DoD'
df1['source_type'] = 'drinking water'

In [7]:
df1['is_detected'] = df1['concentration'] != 0

In [9]:
df1.to_csv('illinois_dw.csv')